In [ ]:
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer
import tqdm

import spacy
import random
nlp = spacy.load("en_core_web_lg")
spacy = nlp

import pickle
import numpy as np

In [ ]:
xsum_filtered_train = load_from_disk("data/xsum_filtered/train")
# xsum_filtered_val = load_from_disk("data/xsum_filtered/val")
# xsum_filtered_test = load_from_disk("data/xsum_filtered/test")
xsum_filtered_train._data = xsum_filtered_train._data.filter(xsum_filtered_train['keep'])
xsum_corrupted_train = xsum_filtered_train.add_column('corrupted_summary', xsum_filtered_train['summary'])
xsum_corrupted_train = xsum_corrupted_train.add_column('corrupted_flag', ['False']* len(xsum_corrupted_train))

# xsum_filtered_val._data = xsum_filtered_val._data.filter(xsum_filtered_val['keep'])
# xsum_corrupted_val = xsum_filtered_val.add_column('corrupted_summary', xsum_filtered_val['summary'])

# xsum_filtered_test._data = xsum_filtered_test._data.filter(xsum_filtered_test['keep'])
# xsum_corrupted_test = xsum_filtered_test.add_column('corrupted_summary', xsum_filtered_test['summary'])

In [ ]:
# add noise function

NOISE_PROB = 1
DELETE_PROB = 0.8
def addnoise(summary, augmentation_span):
        summary_tokens = [token.text_with_ws for token in summary]

        new_summary = []
        for ix, token in enumerate(summary_tokens):
            # don't modify text inside an augmented span
            apply_augmentation = True
            if augmentation_span:
              for aug_span in augmentation_span:
                if aug_span:
                  span_start, span_end = aug_span
                  if span_start <= ix <= span_end:
                      apply_augmentation = False

            # decide whether to add noise
            if apply_augmentation and random.random() < NOISE_PROB:
                # decide whether to replicate or delete token
                if random.random() < DELETE_PROB:
                    # update spans and skip token
                    if augmentation_span:
                      for el in range(0, len(augmentation_span)):
                        aug_span = augmentation_span[el]
                        if aug_span:
                          span_start, span_end = aug_span
                          if ix < span_start:
                            span_start -= 1
                            span_end -= 1
                          aug_span = span_start, span_end
                          augmentation_span[el] = aug_span
                      if len(new_summary) > 0:
                        if new_summary[-1][-1] != " ":
                          new_summary[-1] = new_summary[-1] + " "
                      continue      
                else:
                  if augmentation_span:
                    for el in range(0, len(augmentation_span)):
                      aug_span = augmentation_span[el]
                      if aug_span:
                        span_start, span_end = aug_span
                        if ix < span_start:
                          span_start += 1
                          span_end += 1
                        aug_span = span_start, span_end
                        augmentation_span[el] = aug_span

                  new_summary.append(token)
            new_summary.append(token)
        new_summary = spacy("".join(new_summary))

        if summary.text == new_summary.text:
#             return new_summary
            return new_summary
        else:
            return new_summary
        
docs_to_noise = xsum_corrupted_train['id']
docs_to_noise = np.random.choice(docs_to_noise, int(0.05*len(docs_to_noise)))

In [ ]:
def noise_summary(example):
    if example['id'] in docs_to_noise:
        doc = nlp(example['summary'])
        new_doc = addnoise(doc, [(0, len(doc))])
        example['corrupted_summary'] = new_doc.text
        return example
    else:
        return example

In [ ]:
xsum_corrupted_train = xsum_corrupted_train.map(noise_summary)

In [ ]:
# Comment our perturbations you don't want. We use this to create 

perturbations = [
    "predicate",
    "smart",
    "s_o", 
    "bt", 
#     "prn", 
#     "dat", 
#     "num", 
#     "ent", 
#     "neg"
]

In [ ]:
# Change num_each_perturbation if you want to add more perturbations to train on.
num_each_perturbation = 10000
for pert_type in perturbations:
    
    with open(f'data/corruption_dict/{pert_type}.pkl', 'rb') as f:
        corrupted_summaries = pickle.load(f)
        
    def corrupt_summary(example):
        docid = example['id']
        if docid in corrupted_summaries:
            example['corrupted_summary'] = corrupted_summaries[docid]
            example['corrupted_flag'] = pert_type
            return example
        else:
            return example
        
    filters = list(corrupted_summaries.keys())
    
    
    filters = filters[:num_each_perturbation]
    
    corrupted_summaries = {ids: corrupted_summaries[ids] for ids in filters}
    xsum_corrupted_train = xsum_corrupted_train.map(corrupt_summary)
#     xsum_corrupted_train = xsum_corrupted_train.map(corrupt_summary)
#     xsum_corrupted_train = xsum_corrupted_train.map(corrupt_summary)

    print(f'Added {len(corrupted_summaries)} {pert_type} corruptions to the dataset')

In [ ]:
xsum_corrupted_train.save_to_disk("data/xsum_corrupted_predicateonly/train")
# xsum_corrupted_val.save_to_disk("data/xsum_corrupted/val")
# xsum_corrupted_test.save_to_disk("data/xsum_corrupted/test")

Entity only perturbations

In [ ]:
xsum_filtered_train = load_from_disk("data/xsum_filtered/train")
# xsum_filtered_val = load_from_disk("data/xsum_filtered/val")
# xsum_filtered_test = load_from_disk("data/xsum_filtered/test")
xsum_filtered_train._data = xsum_filtered_train._data.filter(xsum_filtered_train['keep'])
xsum_corrupted_train = xsum_filtered_train.add_column('corrupted_summary', xsum_filtered_train['summary'])
xsum_corrupted_train = xsum_corrupted_train.add_column('corrupted_flag', ['False']* len(xsum_corrupted_train))

# xsum_filtered_val._data = xsum_filtered_val._data.filter(xsum_filtered_val['keep'])
# xsum_corrupted_val = xsum_filtered_val.add_column('corrupted_summary', xsum_filtered_val['summary'])

# xsum_filtered_test._data = xsum_filtered_test._data.filter(xsum_filtered_test['keep'])
# xsum_corrupted_test = xsum_filtered_test.add_column('corrupted_summary', xsum_filtered_test['summary'])

In [ ]:
# Comment our perturbations you don't want.

perturbations = [
#     "predicate",
#     "smart",
#     "s_o", 
    "bt", 
    "prn", 
    "dat", 
    "num", 
    "ent", 
    "neg"
]

In [ ]:
# Change num_each_perturbation if you want to add more perturbations to train on.
num_each_perturbation = 8000
for pert_type in perturbations:
    
    with open(f'data/corruption_dict/{pert_type}.pkl', 'rb') as f:
        corrupted_summaries = pickle.load(f)
        
    def corrupt_summary(example):
        docid = example['id']
        if docid in corrupted_summaries:
            example['corrupted_summary'] = corrupted_summaries[docid]
            example['corrupted_flag'] = pert_type
            return example
        else:
            return example
        
    filters = list(corrupted_summaries.keys())
    
    
    filters = filters[:num_each_perturbation]
    
    corrupted_summaries = {ids: corrupted_summaries[ids] for ids in filters}
    xsum_corrupted_train = xsum_corrupted_train.map(corrupt_summary)

    print(f'Added {len(corrupted_summaries)} {pert_type} corruptions to the dataset')

In [ ]:
xsum_corrupted_train.save_to_disk("data/xsum_corrupted_entityonly/train")